In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
np.random.seed(1)



In [2]:
arr_scale = 60 # 60 minutes
staff_scale = 20 # 20 minutes

arrival_times = np.cumsum(np.random.exponential(scale=arr_scale, size=90000))
arrival_times = arrival_times.astype(int)
print(arrival_times[0:10])

# This makes arrivals indep of anything.
arrival_dict = defaultdict(bool)
for arr in arrival_times:
    arrival_dict[arr] = True

[ 32 108 108 130 139 145 158 183 213 260]


In [3]:
state_tuple = (0,0,0) # customers at stations 1,2,3
state_dict = defaultdict(int)
time = 0
doc_busy_time = 0
nurse_busy_time = 0
shared_busy_time = 0
max_arr = np.max(arrival_times)
while time < max_arr:
    # If a customer arrives at time
    if arrival_dict[time]:
        next_state_tuple = (state_tuple[0]+1, state_tuple[1], state_tuple[2])
        state_dict[(state_tuple, next_state_tuple)] += 1
        state_tuple = next_state_tuple
        time += 1
        continue # this restarts while loop to the top

    # If both doc and nurse are busy at station 1
    if shared_busy_time > 0:
        shared_busy_time -= 1
        if shared_busy_time == 0:
            next_state_tuple = (state_tuple[0]-1, state_tuple[1]+1, state_tuple[2])
            state_dict[(state_tuple, next_state_tuple)] += 1
            state_tuple = next_state_tuple
            time += 1
            continue

    # If nurse is busy, and NOT at station 1
    if nurse_busy_time > 0 and shared_busy_time == 0:
        nurse_busy_time -= 1
        if nurse_busy_time == 0:
            next_state_tuple = (state_tuple[0], state_tuple[1]-1, state_tuple[2]+1)
            state_dict[(state_tuple, next_state_tuple)] += 1     
            state_tuple = next_state_tuple
            time += 1 
            continue

    # If doc is busy, and NOT at station 1
    if doc_busy_time > 0 and shared_busy_time == 0:
        doc_busy_time -= 1
        if doc_busy_time == 0:
            next_state_tuple = (state_tuple[0], state_tuple[1], state_tuple[2]-1)
            state_dict[(state_tuple, next_state_tuple)] += 1  
            state_tuple = next_state_tuple
            time += 1
            continue
            
        
    if state_tuple[0] > 0:
        if doc_busy_time == 0 and nurse_busy_time == 0 and shared_busy_time == 0:
            shared_busy_time = int(np.random.exponential(scale=staff_scale, size=None))
    elif state_tuple[0] == 0 and state_tuple[1] > 0 and state_tuple[2] > 0:
        if doc_busy_time == 0 and shared_busy_time == 0:
            doc_busy_time = int(np.random.exponential(scale=staff_scale, size=None))
        if nurse_busy_time == 0 and shared_busy_time == 0:
            nurse_busy_time = int(np.random.exponential(scale=staff_scale, size=None))                
    elif state_tuple[0] == 0 and state_tuple[1] == 0 and state_tuple[2] > 0:
        if doc_busy_time == 0 and shared_busy_time == 0:
            doc_busy_time = int(np.random.exponential(scale=staff_scale, size=None))                
    elif state_tuple[0] == 0 and state_tuple[1] > 0 and state_tuple[2] == 0:
        if nurse_busy_time == 0 and shared_busy_time == 0:
            nurse_busy_time = int(np.random.exponential(scale=staff_scale, size=None))  
            
    # most time between t=T to t = T+ 1 no queue / customer in line state changes
    # state_dict[(state_tuple, state_tuple)] += 1  
    # Take this out above, to not consider no changes.
    time += 1
                   
    

    
print('done')        



done


In [4]:
print(list(state_dict.items())[0:30])

[(((0, 0, 0), (1, 0, 0)), 10789), (((1, 0, 0), (0, 1, 0)), 9877), (((0, 1, 0), (0, 0, 1)), 9947), (((0, 0, 1), (0, 0, 0)), 10789), (((1, 0, 0), (2, 0, 0)), 3643), (((2, 0, 0), (3, 0, 0)), 1108), (((3, 0, 0), (2, 1, 0)), 970), (((2, 1, 0), (3, 1, 0)), 782), (((3, 1, 0), (2, 2, 0)), 558), (((2, 2, 0), (3, 2, 0)), 373), (((3, 2, 0), (2, 3, 0)), 339), (((2, 3, 0), (1, 4, 0)), 378), (((1, 4, 0), (2, 4, 0)), 310), (((2, 4, 0), (1, 5, 0)), 262), (((1, 5, 0), (2, 5, 0)), 197), (((2, 5, 0), (1, 6, 0)), 136), (((1, 6, 0), (0, 7, 0)), 107), (((0, 7, 0), (1, 7, 0)), 198), (((1, 7, 0), (2, 7, 0)), 96), (((2, 7, 0), (2, 6, 1)), 68), (((2, 6, 1), (1, 7, 1)), 513), (((1, 7, 1), (0, 8, 1)), 575), (((0, 8, 1), (1, 8, 1)), 203), (((1, 8, 1), (1, 8, 0)), 86), (((1, 8, 0), (1, 7, 1)), 170), (((1, 7, 1), (2, 7, 1)), 270), (((2, 7, 1), (1, 8, 1)), 403), (((1, 8, 1), (0, 9, 1)), 438), (((0, 9, 1), (0, 8, 2)), 434), (((0, 8, 2), (1, 8, 2)), 132)]


In [5]:
#list(state_dict.items())

In [6]:
total = 0
numer = 0
for k,v in state_dict.items():
    if k[0][0] > 0:
        total += 1
        diff0 = k[1][0] - k[0][0]
        diff1 = k[1][1] - k[0][1]
        diff2 = k[1][2] - k[0][2]
        b1 = diff0 == -1
        b2 = diff1 == 1
        b3 = diff2 == 0
        if b1 and b2 and b3:
            numer += 1

trans_str = ['(x1,x2,x3) to (x1-1,x2+1,x3) given x1>0']
prob_list = [numer/total]



In [7]:
total = 0
numer = 0
for k,v in state_dict.items():
    if k[0][0] == 0:
        total += 1
        diff0 = k[1][0] - k[0][0]
        diff1 = k[1][1] - k[0][1]
        diff2 = k[1][2] - k[0][2]
        b1 = diff0 == -1
        b2 = diff1 == 1
        b3 = diff2 == 0
        if b1 and b2 and b3:
            numer += 1

trans_str += ['(x1,x2,x3) to (x1-1,x2+1,x3) given x1==0']
prob_list += [numer/total]

In [8]:
total = 0
numer = 0
for k,v in state_dict.items():
    if k[0][0] == 0:
        total += 1
        diff0 = k[1][0] - k[0][0]
        diff1 = k[1][1] - k[0][1]
        diff2 = k[1][2] - k[0][2]
        b1 = diff0 == 1
        b2 = diff1 == 0
        b3 = diff2 == 0
        if b1 and b2 and b3:
            numer += 1

trans_str += ['(x1,x2,x3) to (x1+1,x2,x3) given x1==0']
prob_list += [numer/total]

In [9]:
total = 0
numer = 0
for k,v in state_dict.items():
    if k[0][0] > 0:
        total += 1
        diff0 = k[1][0] - k[0][0]
        diff1 = k[1][1] - k[0][1]
        diff2 = k[1][2] - k[0][2]
        b1 = diff0 == 1
        b2 = diff1 == 0
        b3 = diff2 == 0
        if b1 and b2 and b3:
            numer += 1

trans_str += ['(x1,x2,x3) to (x1+1,x2,x3) given x1>0']
prob_list += [numer/total]

In [10]:
total = 0
numer = 0
for k,v in state_dict.items():
    if k[0][0] > 0:
        total += 1
        diff0 = k[1][0] - k[0][0]
        diff1 = k[1][1] - k[0][1]
        diff2 = k[1][2] - k[0][2]
        b1 = diff0 == 0
        b2 = diff1 == -1
        b3 = diff2 == 1
        if b1 and b2 and b3:
            numer += 1

trans_str += ['(x1,x2,x3) to (x1,x2-1,x3+1) given x1>0']
prob_list += [numer/total]

total = 0
numer = 0
for k,v in state_dict.items():
    if k[0][0] == 0:
        total += 1
        diff0 = k[1][0] - k[0][0]
        diff1 = k[1][1] - k[0][1]
        diff2 = k[1][2] - k[0][2]
        b1 = diff0 == 0
        b2 = diff1 == -1
        b3 = diff2 == 1
        if b1 and b2 and b3:
            numer += 1

trans_str += ['(x1,x2,x3) to (x1,x2-1,x3+1) given x1==0']
prob_list += [numer/total]

In [11]:
total = 0
numer = 0
for k,v in state_dict.items():
    if k[0][0] > 0:
        total += 1
        diff0 = k[1][0] - k[0][0]
        diff1 = k[1][1] - k[0][1]
        diff2 = k[1][2] - k[0][2]
        b1 = diff0 == 0
        b2 = diff1 == 0
        b3 = diff2 == -1
        if b1 and b2 and b3:
            numer += 1

trans_str += ['(x1,x2,x3) to (x1,x2,x3-1) given x1>0']
prob_list += [numer/total]

total = 0
numer = 0
for k,v in state_dict.items():
    if k[0][0] == 0:
        total += 1
        diff0 = k[1][0] - k[0][0]
        diff1 = k[1][1] - k[0][1]
        diff2 = k[1][2] - k[0][2]
        b1 = diff0 == 0
        b2 = diff1 == 0
        b3 = diff2 == -1
        if b1 and b2 and b3:
            numer += 1

trans_str += ['(x1,x2,x3) to (x1,x2,x3-1) given x1==0']
prob_list += [numer/total]

In [12]:
df = pd.DataFrame()
df['trans_str'] = trans_str
df['trans_probability'] = prob_list
df

,trans_str,trans_probability
0,"(x1,x2,x3) to (x1-1,x2+1,x3) given x1>0",0.380483
1,"(x1,x2,x3) to (x1-1,x2+1,x3) given x1==0",0.000000
2,"(x1,x2,x3) to (x1+1,x2,x3) given x1==0",0.275192
3,"(x1,x2,x3) to (x1+1,x2,x3) given x1>0",0.285612
4,"(x1,x2,x3) to (x1,x2-1,x3+1) given x1>0",0.164595
5,"(x1,x2,x3) to (x1,x2-1,x3+1) given x1==0",0.369634
6,"(x1,x2,x3) to (x1,x2,x3-1) given x1>0",0.169310
7,"(x1,x2,x3) to (x1,x2,x3-1) given x1==0",0.355174
